In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [2]:

ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your Azure Machine Learning workspace
    subscription_id = "f1a8fafd-a8a3-46d8-bb5e-01deb63d275d"
    resource_group = "aml-rg"
    workspace = "aml-testws"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.


In [ ]:
#!pip install pandas
#!pip install pip install scikit-learn
#!pip install mltable
#!pip install mlflow
#!pip install ml-flow-client
#!pip install azureml-mlflow


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load training data from CSV file
training_data = pd.read_csv('./telcocustomerchurn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Separate the data into features (X) and labels (y)
X = training_data.drop(columns=['Churn'])  # Assuming 'label' is the column containing class labels
y = training_data['Churn']

# Split the data into training and test sets while maintaining class balance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Combine the features and labels for training and test sets
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# Save the generated test data to a new CSV file
test_data.to_csv('./telcocustomerchurn/WA_Fn-UseC_-Telco-Customer-Churn_Test.csv', index=False)

# Save the training data to a new CSV file
train_data.to_csv('./telcocustomerchurn/WA_Fn-UseC_-Telco-Customer-Churn_Train.csv', index=False)



In [54]:
test_data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
437,4376-KFVRS,Male,0,Yes,Yes,72,Yes,Yes,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),114.05,8468.2,No
2280,2754-SDJRD,Female,1,No,No,8,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),100.15,908.55,No
2235,9917-KWRBE,Female,0,Yes,Yes,41,Yes,Yes,DSL,Yes,...,Yes,No,Yes,No,One year,Yes,Credit card (automatic),78.35,3211.2,No
4460,0365-GXEZS,Male,0,Yes,No,18,Yes,No,Fiber optic,No,...,Yes,Yes,No,No,Month-to-month,No,Electronic check,78.20,1468.75,No
3761,9385-NXKDA,Female,0,Yes,No,72,Yes,Yes,DSL,Yes,...,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),82.65,5919.35,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5143,5204-HMGYF,Female,0,Yes,Yes,49,Yes,No,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,87.20,4345,No
4439,9950-MTGYX,Male,0,Yes,Yes,28,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),20.30,487.95,No
3857,3675-EQOZA,Male,0,No,No,5,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Bank transfer (automatic),20.65,93.55,No
4758,3646-ITDGM,Female,0,No,No,56,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),19.70,1051.9,No


In [53]:
train_data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
3738,4950-BDEUX,Male,0,No,No,35,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,49.20,1701.65,No
3151,7993-NQLJE,Male,0,Yes,Yes,15,Yes,No,Fiber optic,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,75.10,1151.55,No
4860,7321-ZNSLA,Male,0,Yes,Yes,13,No,No phone service,DSL,Yes,...,No,Yes,No,No,Two year,No,Mailed check,40.55,590.35,No
3867,4922-CVPDX,Female,0,Yes,No,26,Yes,No,DSL,No,...,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),73.50,1905.7,No
3810,2903-YYTBW,Male,0,Yes,Yes,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,44.55,44.55,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6303,6308-CQRBU,Female,0,Yes,No,71,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Two year,No,Electronic check,109.25,7707.7,No
6227,2842-JTCCU,Male,0,No,No,2,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Bank transfer (automatic),46.05,80.35,Yes
4673,6402-ZFPPI,Female,1,No,No,25,Yes,Yes,Fiber optic,Yes,...,No,No,Yes,Yes,Month-to-month,Yes,Mailed check,102.80,2660.2,Yes
2710,3594-BDSOA,Female,0,Yes,No,24,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Credit card (automatic),20.40,482.8,No


In [19]:
import mltable
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes


train_data_paths = [
    {'file': './wa_telco_customer_churn_train_data/WA_Fn-UseC_-Telco-Customer-Churn_Train.csv'},
]

train_table = mltable.from_delimited_files(train_data_paths)
train_table.save('./wa_telco_customer_churn_train_data')
# Save the training data to a new CSV file
train_data.to_csv(train_data_paths[0]['file'], index=False)


test_data_paths = [
    {'file': './wa_telco_customer_churn_test_data/WA_Fn-UseC_-Telco-Customer-Churn_Test.csv'},
]

test_table = mltable.from_delimited_files(test_data_paths)
test_table.save('./wa_telco_customer_churn_test_data')
# Save the generated test data to a new CSV file
test_data.to_csv(test_data_paths[0]['file'], index=False)



In [23]:
def create_dataset(data_path, data_asset_name, data_asset_version):
    data_asset_def = Data(
        name=data_asset_name,
        version=data_asset_version,
        description=data_asset_name,
        path=data_path,
        type=AssetTypes.MLTABLE,
    )
    data_asset = None
    ## create data asset if it doesn't already exist:
    try:
        data_asset = ml_client.data.get(name=data_asset_name, version=data_asset_version)
        print(
            f"Data asset already exists. Name: {data_asset_def.name}, version: {data_asset_def.version}"
        )
    except:
        ml_client.data.create_or_update(data_asset_def)
        print(f"Data asset created. Name: {data_asset_def.name}, version: {data_asset_def.version}")

    data_asset = ml_client.data.get(name=data_asset_name, version=data_asset_version)
    return data_asset

In [24]:
data_path = "./wa_telco_customer_churn_train_data"
data_asset_name = "wa_telco_customer_churn_train_data"
data_asset_version = "1.5"
train_data_asset = create_dataset(data_path, data_asset_name, data_asset_version)

Data asset already exists. Name: wa_telco_customer_churn_train_data, version: 1.5


In [17]:
data_path = "./wa_telco_customer_churn_test_data"
data_asset_name = "wa_telco_customer_churn_test_data"
data_asset_version = "1.5"
create_dataset(data_path, data_asset_name, data_asset_version)


Data asset created. Name: wa_telco_customer_churn_test_data, version: 1.5


'azureml://subscriptions/f1a8fafd-a8a3-46d8-bb5e-01deb63d275d/resourcegroups/aml-rg/workspaces/aml-testws/datastores/workspaceblobstore/paths/LocalUpload/315e26e265c67b1f96f84514ccb01fe9/wa_telco_customer_churn_test_data/'

In [25]:
from azure.ai.ml import automl, Input

training_data_input = Input(
    type=AssetTypes.MLTABLE, path=train_data_asset.path
)


# configure the classification job
classification_job = automl.classification(
    compute="cpu-cluster",
    experiment_name="wa-telco-customer-churn-classification",
    training_data=training_data_input,
    target_column_name="Churn",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"my_custom_tag": "My custom value"}
    
)

# Limits are all optional
classification_job.set_limits(
    timeout_minutes=600, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# Training properties are optional
classification_job.set_training(
    allowed_training_algorithms=["GradientBoosting", "DecisionTree", "LightGBM" , "RandomForest"], 
    enable_onnx_compatible_models=True
)

classification_job.set_featurization(
    mode="auto",
)

In [26]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

# Get a URL for the status of the job
returned_job.services["Studio"].endpoint

Created job: compute: azureml:cpu-cluster
creation_context:
  created_at: '2023-09-03T21:54:14.091242+00:00'
  created_by: Anil Dwarakanath
  created_by_type: User
display_name: bubbly_kiwi_srpvk26ch0
experiment_name: wa-telco-customer-churn-classification
featurization:
  enable_dnn_featurization: false
  mode: auto
id: azureml:/subscriptions/f1a8fafd-a8a3-46d8-bb5e-01deb63d275d/resourceGroups/aml-rg/providers/Microsoft.MachineLearningServices/workspaces/aml-testws/jobs/bubbly_kiwi_srpvk26ch0
limits:
  enable_early_termination: true
  max_concurrent_trials: 1
  max_cores_per_trial: -1
  max_nodes: 1
  max_trials: 5
  timeout_minutes: 600
  trial_timeout_minutes: 20
log_verbosity: info
n_cross_validations: 5
name: bubbly_kiwi_srpvk26ch0
outputs: {}
primary_metric: accuracy
properties:
  azureml.git.dirty: 'True'
  mlflow.source.git.branch: main
  mlflow.source.git.commit: 604157f9a649655d373442a5fea892d8238000a1
  mlflow.source.git.repoURL: https://github.com/anildwarepo/CustomerChurnP

'https://ml.azure.com/runs/bubbly_kiwi_srpvk26ch0?wsid=/subscriptions/f1a8fafd-a8a3-46d8-bb5e-01deb63d275d/resourcegroups/aml-rg/workspaces/aml-testws&tid=7f1290b4-3c39-4277-a63e-c577680a12cf'

In [27]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://74f7aea0-6b0e-4b75-a8e4-001d9a929102.workspace.westus.api.azureml.ms/mlflow/v1.0/subscriptions/f1a8fafd-a8a3-46d8-bb5e-01deb63d275d/resourceGroups/aml-rg/providers/Microsoft.MachineLearningServices/workspaces/aml-testws


In [28]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://74f7aea0-6b0e-4b75-a8e4-001d9a929102.workspace.westus.api.azureml.ms/mlflow/v1.0/subscriptions/f1a8fafd-a8a3-46d8-bb5e-01deb63d275d/resourceGroups/aml-rg/providers/Microsoft.MachineLearningServices/workspaces/aml-testws


In [29]:
from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

# Initialize MLFlow client
mlflow_client = MlflowClient()

In [30]:

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(returned_job.name)
best_child_run_id = mlflow_parent_run.data.tags['automl_best_child_run_id']
# get the best child run
best_run = mlflow_client.get_run(best_child_run_id)

In [31]:
best_run

<Run: data=<RunData: metrics={'AUC_macro': 0.8450202375216364,
 'AUC_micro': 0.8894772873981246,
 'AUC_weighted': 0.8450202375216364,
 'accuracy': 0.8004992899932388,
 'average_precision_score_macro': 0.8012384954071552,
 'average_precision_score_micro': 0.8909891875656996,
 'average_precision_score_weighted': 0.8646616920011929,
 'balanced_accuracy': 0.7049153222348113,
 'f1_score_macro': 0.7204458800520084,
 'f1_score_micro': 0.8004992899932388,
 'f1_score_weighted': 0.7906582730652983,
 'log_loss': 0.4165229529734602,
 'matthews_correlation': 0.45366616498197904,
 'norm_macro_recall': 0.40983064446962264,
 'precision_score_macro': 0.7515356600903705,
 'precision_score_micro': 0.8004992899932388,
 'precision_score_weighted': 0.790768753542635,
 'recall_score_macro': 0.7049153222348113,
 'recall_score_micro': 0.8004992899932388,
 'recall_score_weighted': 0.8004992899932388,
 'weighted_accuracy': 0.8614858246439322}, params={}, tags={'mlflow.parentRunId': 'bubbly_kiwi_srpvk26ch0',
 'ml

In [32]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azure.ai.ml.constants import ModelType

model_name = "wa_telco_customer_churn_model_best"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="wa_telco_customer_churn_model_best",
    type=AssetTypes.MLFLOW_MODEL,
)

# for downloaded file
# model = Model(path="artifact_downloads/outputs/model.pkl", name=model_name)

registered_model = ml_client.models.create_or_update(model)

In [37]:
# Let's pick the latest version of the model
latest_model = max(
    [(m.version) for m in ml_client.models.list(name=registered_model.name)]
)

print(latest_model)

2


In [38]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [39]:
# Download run's artifacts/outputs
local_path = download_artifacts(
    run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Artifacts downloaded in: c:\source\repos\aml-customer-churn-prediction\artifact_downloads\outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'generated_code', 'internal_cross_validated_models.pkl', 'mlflow-model', 'model.onnx', 'model.pkl', 'model_onnx.json', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']
